In [ ]:
import numpy as np
import scipy.optimize

### PART A & B
### Decide which mirrors to use and ensure that it's stable
print("Part A and B")

lamda = 633e-9  # Wavelength of laser
c = 3e8  # Speed of light (m/s)
n = 1    # Refractive index of air
r = 0.99 # Reflectivity of the mirrors

# Define a stable cavity by equation 32 in Campbell, Optical Cavities
def check_stable(L, R1, R2):
  η = (1 - L/R1)*(1 - L/R2)
  if (0 <= η <= 1):
    print(f"Stable cavity with params:")
    print(f"L = {L:.3e}")
    print(f"R1 = {R1:.3e}")
    print(f"R2 = {R2:.3e}")
  else:
    print("Unstable cavity")
    exit()

# Adjust these parameters to find stable cavity that is reasonable to build
L = 0.55  # Path length (m)
R1 = 100 # Radius of curvature of plane mirror (m), theoretically ∞
R2 = 1.0 # Radius of curvature of  concave mirror (m)
f2 = R2/2 # Focal length of the concave mirror

check_stable(L, R1, R2)

# Design:
# Dimensions 10 inches by 6 inches
# Backside mirror at lower left and lower right
# Concave mirror at upper left
# Planar mirror at upper right
# Path length 55 cm

### PART C
### Calculate the cavity waist, wo, and its location relative to the input mirror
print("Part C")

w0 = np.sqrt(lamda/((np.pi * n)*np.sqrt((R1-L)*(R1+R2-L)/(L*(R2-L))))*R1)
print(f"Cavity waist: {w0:.3e}")
b0 = np.pi*w0**2 /lamda
z = f2/(1+(f2/b0)**2)
print(f"Cavity waist location: {z:.3e}")

### PART D
### Calculate the cavity Free Spectral Range
print("Part D")
FSR = c / (2 * L)
print(f"Free spectral range: {FSR:.3e}")

### PART E
### Calculations
print("Part E")
F = np.pi * np.sqrt(r) / (1 - r)
print(f"Finesse: {F:.3e}")

f1 = c / (2 * L)
print(f"Resonance frequency: {f1:.3e}")

delta_f = FSR / F
print(f"Linewidth: {delta_f:.3e} Hz")

Q = f1 / delta_f
print(f"Quality factor: {Q:.3e}")

n0 = 1 / (2*(1-r**2)) #
print(f"Bounce number: {n0:.3e}")

### PART 2 ###
### for the given values of our laser, we define:
print("Part 2 Optimization of Telescope")
w0_laser = 1e-3  # Laser waist (m)
z_laser = -0.89  # Position of laser waist relative to the cavity input (m)
w0_cavity = w0 # Already calculated cavity waist (m)
z_cavity = L + z  # Cavity waist position relative to the input mirror (m)

# Calculate q-parameters for both
q_laser = z_laser + 1j * (np.pi * w0_laser**2 / lamda)
q_cavity = z_cavity + 1j * (np.pi * w0_cavity**2 / lamda)
print(f"Q-value of laser: {q_laser}")
print(f"Q-value of cavity: {q_cavity}")

# ABCD matrix functions
def lens(f):
    return np.array([[1, 0], [-1/f, 1]])

def space(d):
    return np.array([[1, d], [0, 1]])

def mirror(R):
    return np.array([[1, 0], [-2/R, 1]])

# Define helper function to simulate the beam propogation
def propagate(q, system):
    A, B, C, D = system[0, 0], system[0, 1], system[1, 0], system[1, 1]
    return (A * q + B) / (C * q + D)

# Define optimization function to use in scipy minimize
def mode_matching_min(x):
    f1, d1, f2, d2 = x
    d3 = 1 - d1 - d2
    system = np.linalg.multi_dot([space(z),mirror(R2),space(L), space(d3), lens(f2), space(d2), lens(f1), space(d1)])
    q_out = propagate(q_laser, system)
    return np.abs(q_out - q_cavity)

# Initial guess for optimization:
x0 = [0.2, 0.2, 0.2, 0.2]  # f1, d1, f2, d2

# Define bounds for the parameters:
bounds = [(0.05, 1), (0.05, 1), (0.05, 1), (0.05, 1)]

# Minimize difference between q out and q of cavity
result = scipy.optimize.minimize(mode_matching_min, x0, bounds=bounds)
# Extract the optimized parameters
f1_optimized, d1_optimized, f2_optimized, d2_optimized = result.x
d3_opt = 1 - d1_optimized - d2_optimized

system_optimized = np.linalg.multi_dot([space(z),mirror(R2), space(L), space(d3_opt), lens(f2_optimized), space(d2_optimized), lens(f1_optimized), space(d1_optimized)])
q_out_optimized = propagate(q_laser, system_optimized)

print("Overall maximization of system without fixed focal length:")
print(f"Distance from laser to lens 1 (d1): {d1_optimized:.2f} m")
print(f"Distance between lenses (d2): {d2_optimized:.2f} m")
print(f"Distance from lens 2 to the cavity (d3): {d3_opt:.2f} m")
print(f"\nOptimized q-parameter of the laser just before entering the cavity: {q_out_optimized:.2e}")

### We need to adjust the params from this optimized solution to be practical for the components we have in the lab
# Repeat optimization for fixed focal lengths of the lenses
f1 = 0.2  # Focal length of lens 1 (m)
f2 = 0.1  # Focal length of lens 2 (m)

# Define a new optimization function that only changes d1 and d2
def mode_matching_min2(x):
    d1, d2 = x
    d3 = 1 - d1 - d2
    system = np.linalg.multi_dot([space(z),mirror(R2),space(L), space(d3), lens(f2), space(d2), lens(f1), space(d1)])
    q_out = propagate(q_laser, system)
    return np.abs(q_out - q_cavity)

# Initial guess for d1 and d2 (make sure this outputs a reasonable optimization)
x0 = [0.2, 0.2]

bounds = [(0.1, 1), (0.2, 1)]

# Run the optimization for fixed focal lengths
result = scipy.optimize.minimize(mode_matching_min2, x0, bounds=bounds)

# Extract the optimized parameters
d1_optimized2, d2_optimized2 = result.x
d3_optimized2 = 1 - d1_optimized2 - d2_optimized2

# Propagate the laser through the optimized system
system_optimized2 = np.linalg.multi_dot([space(z),mirror(R2), space(L), space(d3_opt), lens(f2), space(d2_optimized), lens(f1), space(d1_optimized)])
q_out_optimized2 = propagate(q_laser, system_optimized2)


print("After fixing focal length of lenses to 100mm and 200mm:")
print(f"Optimized distance from laser to lens 1 (d1): {d1_optimized:.2f} m")
print(f"Optimized distance between lenses (d2): {d2_optimized:.2f} m")
print(f"Optimized distance from lens 2 to the cavity (d3): {d3_opt:.2f} m")
print(f"Optimized q-parameter of the laser just before entering the cavity: {q_out_optimized2:.2e}")



Part A and B
Stable cavity with params:
L = 5.500e-01
R1 = 1.000e+02
R2 = 1.000e+00
Part C
Cavity waist: 3.167e-04
Cavity waist location: 2.489e-01
Part D
Free spectral range: 2.727e+08
Part E
Finesse: 3.126e+02
Resonance frequency: 2.727e+08
Linewidth: 8.725e+05 Hz
Quality factor: 3.126e+02
Bounce number: 2.513e+01
Part 2 Optimization of Telescope
Q-value of laser: (-0.89+4.963021569652121j)
Q-value of cavity: (0.7988718751025183+0.4977488179739535j)
Overall maximization of system without fixed focal length:
Distance from laser to lens 1 (d1): 0.05 m
Distance between lenses (d2): 0.30 m
Distance from lens 2 to the cavity (d3): 0.65 m

Optimized q-parameter of the laser just before entering the cavity: 7.99e-01+4.98e-01j
After fixing focal length of lenses to 100mm and 200mm:
Optimized distance from laser to lens 1 (d1): 0.05 m
Optimized distance between lenses (d2): 0.30 m
Optimized distance from lens 2 to the cavity (d3): 0.65 m
Optimized q-parameter of the laser just before entering